# Описание задачи
## В задачи пректа входит:
- Нахождение данных для обученяи модели: парсинг и/или иные источники;
- Предобработка полученных данных
- Создание новых признаков с использованием полученных данных;
- Отбор признаков;
- Отбор моделей;
- Обучение и тестирование отобранных моделей на отобранных признаках;
- Подбор макропараметров модели на отобранных признаках;
- Получение предсказанных моделью значений, подготовка и отправка submission

# Импорт библиотек, установка параметров, определение функций
## Импорт бибилиотек

In [1]:
import pandas as pd
import numpy as np
import json 
from datetime import datetime
from datetime import date

from pprint import pprint

import warnings
import os

## Установка параметров

In [2]:
%pylab inline

warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 70)    # выведем больше строк
pd.set_option('display.max_columns', 30) # выведем больше колонок

# альтернативные пути для kaggle и локальный (НЕ ЗАБЫВАТЬ МЕНЯТЬ!)
path = './Project_5_data/'
# path = '/kaggle/input/.....'

# фиксация randomstate
RANDOM_SEED = 42

Populating the interactive namespace from numpy and matplotlib


C:\Users\Samsung\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


## Определение функций

In [3]:
def describe_df(df):
    '''
    Вывод простой статистки DataFrame
    '''
    desc = df.describe(include='all',percentiles=[0.5]).drop(index=['mean','std','50%']).T 
    desc['type'] = [type(x) for x in df.iloc[0]]
    desc['NaN_prop'] = round(df.isna().sum()/len(df),3)
    desc.unique = df.nunique()
    desc.top = df.mode(axis=0).iloc[0]
    desc.top = [str(x)[:30] for x in df.mode(axis=0).iloc[0]]

    desc.freq = [df[col].value_counts().iloc[0] for col in  df.columns]
    
    return desc

def cat_visualisation():
    pass

def num_visualisation():
    pass

# Изучение  валидационных данных 

Что бы понимать какие признаки нам нужны для обучения модели нам необходимо узнать какие признаки описывают наблюдения, для которых мы должны предсказать целевую переменную.  

Поэтому для начала исследуем набор данных эти наблюдения описывающие.

In [4]:
df_test = pd.read_csv(f'{path}test.csv')

### Общий обзор

In [5]:
df_test.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Columns: 32 entries, bodyType to Таможня
dtypes: int64(6), object(26)
memory usage: 8.5+ MB


In [6]:
describe_df(df_test)

,count,unique,top,freq,min,max,type,NaN_prop
bodyType,34686,24,седан,13147,NaN,NaN,<class 'str'>,0.000
brand,34686,12,BMW,4473,NaN,NaN,<class 'str'>,0.000
car_url,34686,34686,https://auto.ru/cars/used/sale,1,NaN,NaN,<class 'str'>,0.000
color,34686,16,чёрный,11010,NaN,NaN,<class 'str'>,0.000
complectation_dict,6418,2364,"{""id"":""4562904"",""name"":""Elegan",51,NaN,NaN,<class 'float'>,0.815
description,34686,31732,Выгода до 82 000 руб. при обме,264,NaN,NaN,<class 'str'>,0.000
engineDisplacement,34686,55,2.0 LTR,7391,NaN,NaN,<class 'str'>,0.000
enginePower,34686,315,249 N12,1708,NaN,NaN,<class 'str'>,0.000
equipment_dict,24690,23705,"{""leather"":true}",108,NaN,NaN,<class 'str'>,0.288
fuelType,34686,5,бензин,28601,NaN,NaN,<class 'str'>,0.000


Всего в исходном валидационном датасете 32 колонки с разными типами данных (6 int, 26 object) и разыми типами признаков:   
-бинарные (в колонках `vendor`, `ПТС`, `Руль`);   
-категориальные (например в колонках `bodyType`, `brand`, `color`, `Владельцы`);  
-количественные (например в `enginePower` или `mileage`).  

Некоторын колонки содержат словари или длинные строки, содержащие сразу несколько признаков. 

Колонка `brand` содержит 12 уникальных значений. Это позволяет нам существенно сократить обьем работы по парсингу, ведь релевантными будут только данные по этим 12 брендам, поэтому только по ним и будем собирать информацию.

Колонка `car_url` содержит ссылки на обьявления о продаже авто на сайте ***auto.ru***, что как бы намекает, что валидационный набор спарсен с сайта ***auto.ru***. Воспользуемся этим же источником.

Три колонки  `priceCurrency`, `Состояние` и `Таможня` содержат только одно уникальное хначение, они ненесут никакой информации, поэтому исключим их из рассмотрения.  
  
Теперь следует ознакомиться с  каждым признак/колонку более детально. Это позволит нам понять в каком виде/формате содержится информация в колонках и соответсвенно сформировать набор данных для обучения в том же виде/формате. Так же детальное ознакомлене позволит окончательно определится с необходимость включения информации из колонок в рассмотрение моделью. В случае включения данных в рассмотрение моделью, провеем EDA. 

### Детальное ознакомление
### bodyType

In [7]:
df_test.bodyType.sample(3)

27638    внедорожник 5 дв.
31597              минивэн
24291        хэтчбек 5 дв.
Name: bodyType, dtype: object

In [8]:
df_test.bodyType.unique()

array(['лифтбек', 'внедорожник 5 дв.', 'хэтчбек 5 дв.', 'седан',
       'компактвэн', 'универсал 5 дв.', 'пикап одинарная кабина',
       'хэтчбек 3 дв.', 'купе', 'кабриолет', 'минивэн',
       'пикап двойная кабина', 'внедорожник 3 дв.', 'родстер', 'микровэн',
       'седан 2 дв.', 'купе-хардтоп', 'фастбек', 'тарга',
       'внедорожник открытый', 'лимузин', 'пикап полуторная кабина',
       'седан-хардтоп', 'фургон'], dtype=object)

Колонка содержит наименование типа кузова в виде строковой величины. Принимается в пассмоттение. 

### brand

In [9]:
df_test.brand.sample(3)

31171       SKODA
885         SKODA
16745    MERCEDES
Name: brand, dtype: object

In [10]:
df_test.brand.unique()

array(['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
       'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI'],
      dtype=object)

Колонка содержит наименование марки в виде строковой величины. Принимается в рассмотрение.

### car_url

In [11]:
df_test.car_url.sample(5)

18351    https://auto.ru/cars/used/sale/mercedes/gl_kla...
26947    https://auto.ru/cars/used/sale/mitsubishi/outl...
20362    https://auto.ru/cars/used/sale/toyota/corolla/...
11153    https://auto.ru/cars/used/sale/nissan/juke/110...
32350    https://auto.ru/cars/used/sale/mitsubishi/lanc...
Name: car_url, dtype: object

В этой колонке содержаться ссылки на обьявления о продаже, послужившие источником при парсинге `test`. В настоящий момент скорее всего ссылки уже не активны, потому колокна будет исключена из рассмотрения моделью. Но нформация в ней содержащаяся позволит нам проверить не попадают ли в тренировочный набор данных наблюдения из валидационного. 

### color

In [12]:
df_test.color.sample(3)

8329     коричневый
8543        голубой
16193    коричневый
Name: color, dtype: object

In [13]:
df_test.color.unique()

array(['синий', 'чёрный', 'серый', 'коричневый', 'белый', 'пурпурный',
       'бежевый', 'серебристый', 'красный', 'зелёный', 'жёлтый',
       'голубой', 'оранжевый', 'фиолетовый', 'золотистый', 'розовый'],
      dtype=object)

Колонка содержит название цвета кузова, принимется в рассмотрение. 

### complectation_dict

In [14]:
df_test.complectation_dict.head(3)

0                                                  NaN
1                                                  NaN
2    {"id":"20026336","name":"Ambition","available_...
Name: complectation_dict, dtype: object

In [15]:
df_test.complectation_dict.loc[2]

'{"id":"20026336","name":"Ambition","available_options":["heated-wash-system","airbag-passenger","lock","door-sill-panel","electro-mirrors","mirrors-heat","cooling-box","computer","seat-transformation","wheel-power","fabric-seats","airbag-side","abs","wheel-leather","climate-control-1","esp","adaptive-light","audiopreparation","ashtray-and-cigarette-lighter","front-centre-armrest","electro-window-back","16-inch-wheels","body-mouldings","condition","airbag-driver","isofix","aux","electro-window-front","light-sensor","hcc","ptf","rain-sensor","tyre-pressure","audiosystem-cd","front-seats-heat","wheel-configuration2","wheel-configuration1","immo","12v-socket","third-rear-headrest"]}'

Колонка содержит словарь стандартной комплектации с наименованием комплектации и списком обрудования (зачастую стандартного, имеющегося в любом автомобиле, например "12v-socket" или "lock") входящим в такую комплектацию. Доля пррпусков (свыше 0.8), перечисление оборудования присутсвуюшегов любом авто данной марки и модели и наличие колонки `equipment_dict` с гораздо меньшим количеством пропусков позволяет исключить колонку из рассмотрения.

### description

In [16]:
print(df_test.description[0])

Все автомобили, представленные в продаже, проходят тщательную проверку по более 40 параметрам. Предоставляем гарантию юридической чистоты, а так же год технической гарантии на двигатель и КПП. Бесплатный тест-драйв. Возможно оформление автомобиля в кредит!

Преимущества автокредитования:
— Первоначальный взнос от 0%;
— Более 30 кредитных программ;
— Процентная ставка от 6% годовых;
— Срок кредита от 6 месяцев до 7 лет;
— Оформление кредита по двум документам;
— Досрочное погашение без штрафов и комиссий;
— Сумма кредита до 2 млн рублей;
— Оформление КАСКО – по желанию;
— Без справок и поручителей.

Сотрудничаем с 12 аккредитованными и сертифицированными банками РФ, среднее время ожидания решения банка 20–30 минут.

При покупке автомобиля по программе «Trade-in – выгодный обмен» или в кредит, получите дополнительную скидку до 80 000 рублей на данный автомобиль!

Записаться на тест-драйв, а так же получить подробную информацию можно у специалистов автоцентра по указанному номеру телефона

В колонке содержится текст, относящийся скорее к продавцу, чем к конкретному автомобилю. Поэтому на данном этапе ( и вероятно вообще) колонка исключается из рассмотрения.

### engineDisplacement

In [17]:
df_test.engineDisplacement.unique()

array(['1.2 LTR', '1.6 LTR', '1.8 LTR', '2.0 LTR', '1.4 LTR', '1.3 LTR',
       '1.0 LTR', '3.6 LTR', '1.5 LTR', '1.9 LTR', '2.8 LTR', '1.1 LTR',
       '2.5 LTR', '4.2 LTR', '3.0 LTR', '4.0 LTR', '5.9 LTR', '2.7 LTR',
       '3.1 LTR', '2.4 LTR', '5.2 LTR', '3.2 LTR', '4.1 LTR', '6.3 LTR',
       '2.3 LTR', '6.0 LTR', '2.2 LTR', '3.7 LTR', '2.9 LTR', '5.0 LTR',
       '3.3 LTR', '2.1 LTR', '2.6 LTR', ' LTR', '3.5 LTR', '1.7 LTR',
       '0.7 LTR', '4.4 LTR', '4.8 LTR', '5.4 LTR', '6.6 LTR', '4.9 LTR',
       '3.8 LTR', '3.4 LTR', '3.9 LTR', '4.6 LTR', '5.6 LTR', '4.5 LTR',
       '5.5 LTR', '6.2 LTR', '4.7 LTR', '4.3 LTR', '5.8 LTR', '5.3 LTR',
       '5.7 LTR'], dtype=object)

In [18]:
df_test.engineDisplacement.value_counts()[' LTR']

55

Колонка содержит обьем двигателя, принимется в рассмотрение. Следует отметить наличие скрытых пропусков (55 стрк `' LTR'`)

### enginePower

In [19]:
df_test.enginePower.unique()[:100]

array(['105 N12', '110 N12', '152 N12', '200 N12', '102 N12', '150 N12',
       '90 N12', '180 N12', '220 N12', '122 N12', '70 N12', '140 N12',
       '125 N12', '54 N12', '86 N12', '75 N12', '64 N12', '95 N12',
       '260 N12', '170 N12', '80 N12', '68 N12', '160 N12', '115 N12',
       '280 N12', '53 N12', '60 N12', '143 N12', '42 N12', '101 N12',
       '58 N12', '193 N12', '79 N12', '30 N12', '100 N12', '50 N12',
       '163 N12', '225 N12', '420 N12', '211 N12', '245 N12', '560 N12',
       '500 N12', '249 N12', '450 N12', '605 N12', '250 N12', '354 N12',
       '120 N12', '290 N12', '230 N12', '350 N12', '204 N12', '255 N12',
       '340 N12', '177 N12', '272 N12', '372 N12', '210 N12', '130 N12',
       '300 N12', '190 N12', '239 N12', '435 N12', '333 N12', '271 N12',
       '326 N12', '238 N12', '310 N12', '233 N12', '252 N12', '133 N12',
       '460 N12', '520 N12', '400 N12', '525 N12', '367 N12', '265 N12',
       '550 N12', '580 N12', '88 N12', '165 N12', '430 N12', '335 N

Колонка содержит мощность двигателя, принмается в рассмотрение.

### equipment_dict

In [20]:
dict.keys

<method 'keys' of 'dict' objects>

In [21]:
# ttt = df_test.equipment_dict.apply(lambda x: json.loads(x) if x==x else x)

In [22]:
# len(ttt[0])

In [23]:
# ttt.head()

In [24]:
# ttt.apply(lambda x: list(x.keys()) if x==x else []).sum()

In [25]:
df_test.equipment_dict.sample(5)

15195                                                  NaN
14292                                                  NaN
1645     {"engine-proof":true,"asr":true,"esp":true,"ai...
31406    {"esp":true,"usb":true,"multi-wheel":true,"xen...
8035     {"electro-window-back":true,"alloy-wheel-disks...
Name: equipment_dict, dtype: object

In [26]:
# df_test.equipment_dict.apply(lambda x: json.loads(x) if x==x else x).sample(5)

В колонке много пропусков (почти 29%), не пропущенные значения представляют собой текстово епредсьавление словаря с перечислением оснащения автомобиля. В рассмотрнеие моделью будет приняты те пункты оснащения, что будут пересекаться с пунктами оснащения обучающего набора данных.

### fuelType

In [27]:
df_test.fuelType.unique()

array(['бензин', 'дизель', 'гибрид', 'электро', 'газ'], dtype=object)

Колонка содержит тип топлива. Принимается врассмотрнеи.

### image

In [28]:
df_test.image.sample(5)

9201     https://autoru.naydex.net/lICN99c69/9bc7d3Rr7k...
13239    https://autoru.naydex.net/G87gDZg56/28c7a8vqVE...
7112     https://avatars.mds.yandex.net/get-autoru-vos/...
22061    https://avatars.mds.yandex.net/get-autoru-vos/...
18194    https://autoru.naydex.net/mcQh17801/f1b5751dCi...
Name: image, dtype: object

Колонка содержит ссылки на фото или аватар авто, исключается из рассмотрения.

### mileage

In [29]:
df_test.mileage.sample(5)

22094     90000
28839    250000
18655     38000
11490    130000
22432     61500
Name: mileage, dtype: int64

Колонка содержит пробег авто, принмается в рассмотрениею

### modelDate

In [30]:
df_test.modelDate.unique()

array([2013, 2017, 2008, 2009, 2016, 2012, 2015, 2010, 2006, 2000, 2007,
       1994, 2004, 1999, 2005, 1976, 2001, 1969, 1996, 1998, 1989, 1934,
       2014, 2011, 2018, 1986, 1997, 1990, 2019, 2002, 1991, 1987, 1980,
       1982, 1938, 1988, 2003, 1983, 1978, 1979, 1984, 1992, 1995, 1993,
       1985, 1974, 1966, 1977, 1981, 1972, 1968, 1975, 1949, 1937, 1936,
       1973, 1959, 1958, 2020, 1965, 1971, 1904, 1963, 1955, 1951, 1960],
      dtype=int64)

Колонка содержит год начала выпуска модели, принимется в рассмотрние.

### model_info

In [31]:
df_test.model_info.sample(5)

181      {"code":"SUPERB","name":"Superb","ru_name":"Су...
15016    {"code":"G35","name":"G","ru_name":"G","morpho...
30880    {"code":"PASSAT","name":"Passat","ru_name":"Па...
7181     {"code":"S60","name":"S60","ru_name":"S60","mo...
23053    {"code":"ES","name":"ES","ru_name":"ES","morph...
Name: model_info, dtype: object

Колонка содержит наименование модели в трех вариантах. Учитывая наличие `model_name` колонка исключается из рассмотрения

### model_name

In [32]:
df_test.model_name.sample(5)

6258         XC60
24292    MULTIVAN
30537      TIGUAN
23680    MULTIVAN
2316      FELICIA
Name: model_name, dtype: object

Наименование модели. Принимается к рассмотрению.

### name

In [33]:
df_test.name.sample(5)

13979        3.5 CVT (249 л.с.) 4WD
31054        1.8 AMT (152 л.с.) 4WD
29645    35d 3.0d AT (286 л.с.) 4WD
9454         320i 2.2 AT (170 л.с.)
9537      35i 3.0 AT (306 л.с.) 4WD
Name: name, dtype: object

Колонка содержит расширение названия модели, указывающее на обьем двигателя и тип привода. Поскольку эта информация уже получена из других колонок, эта в рассмотрнеие не принимается

### numberOfDoors

In [34]:
df_test.numberOfDoors.unique()

array([5, 4, 2, 3, 0], dtype=int64)

Колонка содержит числовое представление количества дверей, принимется в рассмотрение.

### parsing_unixtime

Колонка судя по названию содержит время сбора информации в валидационный набор данных. Непосредственно для обученяи информация не нужна. Но время сбора поможет внести поправку на изменение коньюктуры(курс валюты и т.д.). 
Посмотрим на диапазон `parsing_unixtime`:

In [35]:
# datetime.utcfromtimestamp(df_test.parsing_unixtime.min())

In [36]:
# datetime.utcfromtimestamp(df_test.parsing_unixtime.max())

In [37]:
date.fromtimestamp(df_test.parsing_unixtime.min()),date.fromtimestamp(df_test.parsing_unixtime.max())

(datetime.date(2020, 10, 19), datetime.date(2020, 10, 26))

Как видим парсинг произведен с 19 по 26 октября 20-го года

### productionDate

In [38]:
df_test.productionDate.unique()

array([2014, 2017, 2012, 2011, 2019, 2018, 2010, 2020, 2016, 2013, 2006,
       2007, 2015, 2005, 2008, 2009, 1997, 2004, 2002, 1987, 2003, 2001,
       1976, 2000, 1998, 1995, 1999, 1993, 1939, 1996, 1984, 1990, 1991,
       1992, 1989, 1982, 1985, 1994, 1938, 1981, 1988, 1983, 1980, 1986,
       1978, 1970, 1979, 1977, 1972, 1975, 1969, 1950, 1953, 1949, 1937,
       1959, 1968, 1936, 1904, 1974, 1967, 1961, 1960, 1965, 1963, 1957,
       1952, 1973, 1948], dtype=int64)

Год производства автомобиля ( наверное наряду с моделью автомобиля самы значимый признак). Принмается в рассмотрнеие.

### sell_id

In [39]:
df_test.car_url[1]

'https://auto.ru/cars/used/sale/skoda/octavia/1100549428-595cadf2/'

In [40]:
df_test.sell_id[1]

1100549428

In [41]:
df_test.sell_id.unique()

array([1100575026, 1100549428, 1100658222, ..., 1101364889, 1101362518,
       1101256068], dtype=int64)

Колонка содержит ***id*** обьявления. Как непосредственный признак не интересен, в рассмотрение моделью не принимается. Но если предположить, что ***id*** отражает хронологический порядок появления обьявления, то, например, можно попробовать при "тонкой настройке" модели придать более новым обьявлениям больший вес при обучении.

### super_gen

In [42]:
df_test.super_gen[0]

'{"id":"10373605","displacement":1197,"engine_type":"GASOLINE","gear_type":"FORWARD_CONTROL","transmission":"ROBOT","power":105,"power_kvt":77,"human_name":"1.2 AMT (105 л.с.)","acceleration":10.5,"clearance_min":155,"fuel_rate":5}'

Колонка содержит ***id*** и параметры автомобиля, которые либо являются по сути атрибутами модели (у всех автомобилей одной модели одинаковый клиренс) либо уже известны нам из других колонок (например тип трансмиссии). В рассмотрнеи не принимается.

### vehicleConfiguration

In [43]:
df_test.vehicleConfiguration.sample(5)

4924              SEDAN MECHANICAL 1.8
1892         ALLROAD_5_DOORS ROBOT 1.2
31255           LIFTBACK AUTOMATIC 1.8
22033    ALLROAD_5_DOORS AUTOMATIC 1.3
32008             SEDAN MECHANICAL 1.8
Name: vehicleConfiguration, dtype: object

Содержит информацию о типе кузова, типе трансмиссии обьеме двигателя количестве дверей. Эта информация доступна в других колонках, поэтому `vehicleConfiguration` в рассмотрение не принимется 

### vehicleTransmission

In [44]:
df_test.vehicleTransmission.unique()

array(['роботизированная', 'механическая', 'автоматическая', 'вариатор'],
      dtype=object)

Колонка содержит в тестовом виде тип трансмиссии. Принимается в рассмотрение.

### vendor

In [45]:
df_test.vendor.unique()

array(['EUROPEAN', 'JAPANESE'], dtype=object)

In [46]:
df_test.groupby('vendor').brand.unique()

vendor
EUROPEAN      [SKODA, AUDI, VOLVO, BMW, MERCEDES, VOLKSWAGEN]
JAPANESE    [HONDA, NISSAN, INFINITI, TOYOTA, LEXUS, MITSU...
Name: brand, dtype: object

Как видим колонка содержит обобщающий признак - принадлежность марки к европейским либо японским. Как подсказывает опыт автовладельца динамика падения цены от срока владения у этих двух групп различается (японцы обычно дешевеют медленнее, особенно Toyota). Посчитаем признак полезным, примем в рассмотрение моделью.

### Владельцы

In [47]:
df_test.Владельцы.unique()

array(['3 или более', '1\xa0владелец', '2\xa0владельца'], dtype=object)

Колонка содержит в текстовом виде три категории количества владельцев авто. Примем в рассмотрение

### Владение

In [48]:
df_test.Владение.unique()[:20]

array([nan, '3 года и 2 месяца', '11 месяцев', '4 года и 1 месяц',
       '2 года и 4 месяца', '1 месяц', '3 года и 10 месяцев',
       '2 года и 6 месяцев', '6 лет и 4 месяца', '2 месяца', '4 месяца',
       '1 год и 10 месяцев', '2 года и 7 месяцев', '7 лет и 11 месяцев',
       '7 лет и 5 месяцев', '1 год и 11 месяцев', '2 года и 1 месяц',
       '7 лет', '4 года и 2 месяца', '2 года и 8 месяцев'], dtype=object)

Колонка содержит время прошедшее с момента выпуска авто. Очевидно линейно зависит о даты выпуска автомобиля - исключается из рассмотрения.

### ПТС

In [49]:
df_test.ПТС.unique()

array(['Оригинал', 'Дубликат', nan], dtype=object)

Колонка содержит (если значение не nan) категорию ПТС. Принимается в рассмотрение.

### Привод

In [50]:
df_test.Привод.unique()

array(['передний', 'полный', 'задний'], dtype=object)

Колонка содержит категорию привода. Принимается в расмотрние.

### Руль

In [51]:
df_test.Руль.unique()

array(['Левый', 'Правый'], dtype=object)

Колонка содержит категрию право- или левосторонности управления . В рассмотрение.
 ## Колонки подлежащие рассмотрению моделью
Итого из валидационного набора данных, содержащихся в `df_test` будем спользовать для построения модели информацию из следующих колонок:



In [52]:
columns_to_handle = ['bodyType', 'brand', 'car_url', 'color', 'engineDisplacement', 'enginePower', 
                      'equipment_dict','fuelType', 'mileage', 'modelDate', 'model_name', 'numberOfDoors', 
                      'productionDate', 'sell_id', 'vehicleTransmission', 'vendor', 
                      'Владельцы', 'ПТС', 'Привод', 'Руль']
useful_columns = ['parsing_unixtime']

In [53]:
test = df_test[columns_to_handle]

## РАЗРАБОТКА

In [54]:
'''tmp = pd.read_csv(f'{path}all_marks.csv')
tmp.columns =pd.Index(['bodyType', 'brand', 'car_url', 'color', 'engineDisplacement',
       'enginePower', 'equipment_dict', 'fuelType', 'mileage', 'modelDate',
       'model_name', 'numberOfDoors', 'productionDate', 'sell_id',
       'vehicleTransmission', 'vendor', 'Владельцы', 'ПТС', 'Привод', 'Руль',
       'offerprice'],
      dtype='object')
tmp.to_csv(f'{path}all_marks.csv',index=False)

tmp = pd.read_csv(f'{path}extra_train.csv')
tmp.columns =pd.Index(['bodyType', 'brand', 'car_url', 'color', 'engineDisplacement',
       'enginePower', 'equipment_dict', 'fuelType', 'mileage', 'modelDate',
       'model_name', 'numberOfDoors', 'productionDate', 'sell_id',
       'vehicleTransmission', 'vendor', 'Владельцы', 'ПТС', 'Привод', 'Руль',
       'offerprice'],
      dtype='object')
tmp.to_csv(f'{path}extra_train.csv',index=False)

tmp = pd.read_csv(f'{path}train.csv')
tmp.columns =pd.Index(['bodyType', 'brand', 'car_url', 'color', 'engineDisplacement',
       'enginePower', 'equipment_dict', 'fuelType', 'mileage', 'modelDate',
       'model_name', 'numberOfDoors', 'productionDate', 'sell_id',
       'vehicleTransmission', 'vendor', 'Владельцы', 'ПТС', 'Привод', 'Руль',
       'offerprice'],
      dtype='object')
tmp.to_csv(f'{path}train.csv',index=False)''';

## =========================================

## Унификация test и train

In [55]:
train = pd.read_csv(f'{path}all_marks.csv')

In [81]:
def categ_diff(column):
    one = train[column]
    another = test[column]
    one_another = set(one.unique()) - set(another.unique())
    another_one = set(another.unique()) - set(one.unique())
    print(one_another)
    print(another_one)

In [57]:
categ_diff('bodyType')

{nan, 'универсал 3 дв.', 'хэтчбек 4 дв.', 'спидстер'}
set()


In [58]:
categ_diff('brand')

{nan, 'MITSUOKA', 'HEINKEL', 'ISUZU', 'WILLYS', 'BRABUS', 'ЗАЗ', 'ГАЗ', 'HUMMER', 'TIANMA', 'PEUGEOT', 'GMC', 'HAIMA', 'MAZDA', 'EXCALIBUR', 'VORTEX', 'GREAT WALL', 'PONTIAC', 'BENTLEY', 'BYD', 'BRILLIANCE', 'FAW', 'ROLLS-ROYCE', 'PACKARD', 'DAIHATSU', 'LIFAN', 'MERCURY', 'GENESIS', 'УАЗ', 'DS', 'SEAT', 'LAND ROVER', 'SUZUKI', 'JAGUAR', 'ZX', 'ACURA', 'ASTON MARTIN', 'CHRYSLER', 'ТАГАЗ', 'RAM', 'EAGLE', 'DERWAYS', 'ГОНОЧНЫЙ АВТОМОБИЛЬ', 'СМЗ', 'DELAGE', 'TESLA', 'KIA', 'SMART', 'PORSCHE', 'TRIUMPH', 'CHERY', 'IRAN KHODRO', 'FERRARI', 'MERCEDES-BENZ', 'GEELY', 'SUBARU', 'HAVAL', 'TATRA', 'PLYMOUTH', 'OLDSMOBILE', 'SCION', 'FIAT', 'JEEP', 'SHANGHAI MAPLE', 'ИЖ', 'MAYBACH', 'DATSUN', 'LANCIA', 'LIGIER', 'GAC', 'CADILLAC', 'RENAULT', 'ASIA', 'DKW', 'MASERATI', 'LINCOLN', 'CHANGAN', 'DACIA', 'SSANGYONG', 'LTI', 'HYUNDAI', 'DAEWOO', 'BORGWARD', 'ZOTYE', 'PROTON', 'ALPINA', 'OPEL', 'ALFA ROMEO', 'RAVON', 'DAIMLER', 'CITROEN', 'ARO', 'SAAB', 'DONGFENG', 'MAHINDRA', 'ROVER', 'AC', 'ЛУАЗ', 'BUIC

In [59]:
'car_url'

'car_url'

In [60]:
categ_diff('color')

{nan}
set()


In [61]:
categ_fiff('engineDisplacement')

{nan, 1.2, 1.6, 1.3, 1.4, 1.5, 2.0, 1.8, 1.9, 3.6, 2.8, 2.3, 3.3, 2.5, 6.0, 6.3, 3.0, 3.5, 4.0, 4.1, 4.4, 4.9, 0.6, 5.9, 5.0, 5.4, 4.6, 5.6, 6.6, 1.0, 5.5, 6.4, 6.5, 7.0, 7.4, 1.1, 6.1, 0.5, 8.2, 8.0, 2.6, 2.1, 3.1, 0.7, 0.2, 1.7, 2.2, 2.7, 3.2, 3.7, 3.8, 0.8, 4.2, 4.8, 4.7, 4.3, 5.2, 5.8, 5.7, 5.3, 6.2, 6.8, 7.3, 7.7, 0.9, 4.5, 2.4, 2.9, 0.4, 3.4, 3.9}
{'5.3 LTR', '5.2 LTR', '2.6 LTR', '2.2 LTR', '6.3 LTR', '2.3 LTR', '1.4 LTR', '4.1 LTR', '4.3 LTR', '3.3 LTR', '2.8 LTR', '4.5 LTR', '1.0 LTR', '6.6 LTR', '2.1 LTR', '6.2 LTR', '5.8 LTR', '4.8 LTR', '5.7 LTR', '2.5 LTR', '3.9 LTR', '5.5 LTR', '4.6 LTR', '3.0 LTR', '2.0 LTR', '4.7 LTR', '4.0 LTR', '1.1 LTR', '2.7 LTR', '2.4 LTR', '3.1 LTR', '3.2 LTR', '1.7 LTR', '3.6 LTR', '4.4 LTR', '3.4 LTR', '3.7 LTR', '2.9 LTR', '1.8 LTR', '5.0 LTR', '3.5 LTR', '0.7 LTR', '4.9 LTR', '5.9 LTR', '5.6 LTR', '3.8 LTR', '5.4 LTR', '4.2 LTR', '1.2 LTR', '1.3 LTR', '1.6 LTR', '1.9 LTR', ' LTR', '1.5 LTR', '6.0 LTR'}


In [62]:
categ_diff('enginePower')

{nan, 10.0, 12.0, 18.0, 19.0, 20.0, 23.0, 25.0, 26.0, 27.0, 29.0, 30.0, 33.0, 34.0, 35.0, 37.0, 38.0, 40.0, 41.0, 42.0, 44.0, 45.0, 46.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 157.0, 158.0, 159.0, 160.0, 161.0, 162.0, 163.0, 164.0, 165.0, 166.0, 167.0, 168.0, 169.0, 170.0, 171.0, 172.0, 173.0, 174.0, 175.0, 176.0, 177.0, 178.

In [63]:
'equipment_dict'

'equipment_dict'

In [82]:
categ_diff('fuelType')

{nan, 'Газ, газобаллонное оборудование', 'Дизель', 'Бензин, газобаллонное оборудование', 'Бензин', 'Дизель, газобаллонное оборудование', 'Гибрид, газобаллонное оборудование', 'Газ', 'Гибрид'}
{'электро', 'гибрид', 'бензин', 'дизель', 'газ'}


In [90]:
'mileage'
# привести к типу int в train

'mileage'

In [87]:
'modelDate'
# привести к дате выпуска в train

'modelDate'

In [92]:
'model_name'
# categ_diff('model_name')
# привести к верхнему регистру в train

'model_name'

In [93]:
categ_diff('numberOfDoors')

{nan}
{0}


In [69]:
'productionDate'

'productionDate'

In [70]:
'sell_id'

'sell_id'

In [71]:
'vehicleTransmission'

'vehicleTransmission'

In [72]:
'vendor'

'vendor'

In [73]:
'Владельцы'

'Владельцы'

In [74]:
'ПТС'

'ПТС'

In [75]:
'Привод'

'Привод'

In [76]:
'Руль'

'Руль'

In [77]:
def unification_test(df):
    
    pass
    return df

## Объединение test и train

## EDA test и train

## Предобработка

## Base-line

## Features construction

## Отбор признаков

## Подбор гиперпараметров

## ========================================================

# Подвал

In [78]:
'''tt = df_test.complectation_dict[2]
ttt = json.loads(tt)
ttt['available_options']'''

"tt = df_test.complectation_dict[2]\nttt = json.loads(tt)\nttt['available_options']"

In [79]:
'''bodyType', 'brand', 'car_url', 'color',
        'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'mileage', 'modelDate', 'model_name',
    'numberOfDoors', 'parsing_unixtime', 
       'productionDate', 'sell_id', 
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль''';

In [80]:
# df_test[df_test.brand == "AUDI"].model_name